In [75]:
import pandas as pd 
from pymongo import MongoClient


In [76]:
df = pd.read_json("journal_api.journals.json")
df_xonobod =df[ df["branch"].apply(lambda x: x["name"] == "Xonobod")]
df_polevoy =df[ df["branch"].apply(lambda x: x["name"] == "Polevoy")]
client = MongoClient("mongodb://localhost:27017/?replicate_set=myReplicaSet")

/Users/aslonkhamidov/Desktop/code/magazin/proposals/.venv/lib/python3.13/site-packages/pymongo/uri_parser_shared.py:328: UserWarning: Unknown option: replicate_set. Did you mean one of (replicaset, retrywrites, uuidrepresentation) or maybe a camelCase version of one? Refer to docstring.
  return get_validated_options(opts, warn)


In [77]:
len(df_xonobod), len(df_polevoy)


(749, 749)

In [78]:
one_operation = df["operations"].iloc[0]
operation_types = df_xonobod["operations"].apply(lambda x:  x[0]["operation_type"] if len(x) >0 and isinstance(x, list) else None).unique()
operation_types

array(['CASH', None, 'CARD', 'TRANSFER', 'transfer', 'cash'], dtype=object)

In [48]:
one_operation

[{'amount': 155000,
  'journal': {'$oid': '64816d617ff10797db3b4ba6'},
  'operation_type': 'CASH',
  'information': 'firma1',
  'time': {'$date': '2023-06-08T11:38:14.959Z'},
  '_id': {'$oid': '6481bda68f196da12d8e98aa'}}]

In [7]:
Transaction = {
  "_id": "62d2b0c8-758b-4b76-a395-454c4041b0d2",
  "transactionbase": {
    "amount": "",
    "description": "10000",
    "type": "credit",
    "payment_method": "cash"
  },
  "type": "supplier",
  "created_at": "",
  "updated_at": "",
  "branch_id": "aadee444-306d-48e1-aa5c-501b86a584be"
}

In [79]:
branches_ids = {
    "xonobod": "aadee444-306d-48e1-aa5c-501b86a584be", 
    "polevoy":"2dbbbb13-60be-425e-b07c-fa4889f0400d"
}


def convert_object_id_str(id):
    if isinstance(id, dict) and "$oid" in id:
        return id["$oid"]
    return id




def convert_operation_to_transaction(branch_name, operation):
    branch_id = branches_ids[branch_name.lower()]
    payment_method = "cash"
    if operation["operation_type"].lower() == "card":
        payment_method = "online_transfer" 
    elif operation["operation_type"].lower() == "transfer":
        payment_method = "online_transfer"
    return {
        "_id": convert_object_id_str(operation["_id"]),
        "transactionbase": {
            "amount": int(operation["amount"]),
            "description": operation["information"],
            "type": "credit",
            "payment_method": payment_method,
        },
        "type": "sales", 
        "created_at": operation["time"]["$date"], 
        "updated_at": operation["time"]["$date"], 
        "branch_id": branch_id,
    }

# journal = {
#   "_id": {
#     "$oid": "685ab4cffad6033ab220b2de"
#   },
#   "branch": {
#     "name": "Xonobod",
#     "location": "Xonobod",
#     "phone": "+998 97 034 38 58",
#     "_id": "aadee444-306d-48e1-aa5c-501b86a584be"
#   },
#   "date": "",
#   "shift_is_closed": False,
#   "terminal_income": 0,
#   "cash_left": 0,
#   "total": "5992000",
#   "operations": [
#    "id"
#   ]
# }


def convert_journal(journal):
    
    branch_name = journal["branch"]["name"]
    operations = [convert_operation_to_transaction(branch_name, operation) for operation in journal["operations"]]
    branch = journal["branch"]
    branch["_id"]  = branches_ids[branch_name.lower()]
    r =   {
        "_id": convert_object_id_str(journal["_id"]),
        "branch" : branch,
        "date": journal["date"]["$date"],
        "shift_is_closed": journal["shift_is_closed"],
        "terminal_income": int(journal["terminal_income"]),
        "cash_left": int(journal["cash_left"]),
        "total": int(journal["total"]),
        "operations": [convert_operation_to_transaction(branch_name,operation) for operation in journal["operations"]],
    }
    
    print(r)
    return r


df_xonobod_new = df_xonobod.apply(convert_journal, axis=1)
df_polevoy_new = df_polevoy.apply(convert_journal, axis=1)

{'_id': '64816d617ff10797db3b4ba6', 'branch': {'name': 'Xonobod', 'location': 'Xonobod', 'phone': '+998 97 034 38 58', '_id': 'aadee444-306d-48e1-aa5c-501b86a584be'}, 'date': '2023-06-08T00:00:00Z', 'shift_is_closed': True, 'terminal_income': 6666, 'cash_left': 6666, 'total': 268332, 'operations': [{'_id': '6481bda68f196da12d8e98aa', 'transactionbase': {'amount': 155000, 'description': 'firma1', 'type': 'credit', 'payment_method': 'cash'}, 'type': 'sales', 'created_at': '2023-06-08T11:38:14.959Z', 'updated_at': '2023-06-08T11:38:14.959Z', 'branch_id': 'aadee444-306d-48e1-aa5c-501b86a584be'}]}
{'_id': '648179417ff10797db3b4bae', 'branch': {'name': 'Xonobod', 'location': 'Xonobod', 'phone': '+998 97 034 38 58', '_id': 'aadee444-306d-48e1-aa5c-501b86a584be'}, 'date': '2023-06-07T00:00:00Z', 'shift_is_closed': True, 'terminal_income': 0, 'cash_left': 0, 'total': 240000, 'operations': [{'_id': '6482d0e68f196da12d8e98ba', 'transactionbase': {'amount': 240000, 'description': 'Click', 'type': 

In [51]:
df_xonobod_new.iloc[5]
df_polevoy_new.iloc[10]

{'_id': '648e782ec83c7f8bd5528c31',
 'branch': {'name': 'Polevoy',
  'location': 'Polevoy Savatchi mahallasi',
  'phone': '+998 33 119 12 13',
  '_id': '2dbbbb13-60be-425e-b07c-fa4889f0400d'},
 'date': '2023-06-18T00:00:00Z',
 'shift_is_closed': True,
 'terminal_income': 1415000,
 'cash_left': 8100000,
 'total': 9733000,
 'operations': [{'_id': '648e785fc83c7f8bd5528c32',
   'transactionbase': {'amount': 63000,
    'description': 'Click ',
    'type': 'credit',
    'payment_method': 'cash'},
   'type': 'sales',
   'created_at': '2023-06-18T03:22:07.612Z',
   'updated_at': '2023-06-18T03:22:07.612Z',
   'branch_id': '2dbbbb13-60be-425e-b07c-fa4889f0400d'},
  {'_id': '648f1169c83c7f8bd5528c47',
   'transactionbase': {'amount': 155000,
    'description': 'Click ',
    'type': 'credit',
    'payment_method': 'cash'},
   'type': 'sales',
   'created_at': '2023-06-18T14:15:05.959Z',
   'updated_at': '2023-06-18T14:15:05.959Z',
   'branch_id': '2dbbbb13-60be-425e-b07c-fa4889f0400d'}]}

In [80]:

def upload_to_mongo(journal):
    # print(journal)
    print(journal["_id"])
    # upload transactions
    
    transactions = journal["operations"]
    if len(transactions) > 0:
        # print("type", type(transactions))
        # print(transactions)
        client["magazin"]["transactions"].insert_many(transactions)
    transactions_ids = [
        transaction["_id"] for transaction in journal["operations"]
    ]
    
    # upload journal
    journal["operations"] = transactions_ids
    client["magazin"]["journals"].insert_one(journal)
df_polevoy_new.apply(upload_to_mongo)
df_xonobod_new.apply(upload_to_mongo)
    
    

64816df17ff10797db3b4ba8
64829cff8f196da12d8e98b5
648381568f196da12d8e98d7
64855c8e8f196da12d8e98fd
6486a768c83c7f8bd5528b84
6487fb46c83c7f8bd5528ba1
6489591fc83c7f8bd5528bc6
648aa2a6c83c7f8bd5528be5
648c38b1c83c7f8bd5528c04
648d3e4dc83c7f8bd5528c1d
648e782ec83c7f8bd5528c31
648fd54ec83c7f8bd5528c4c
64912ad656b78aefd50371f6
6492619356b78aefd503720e
6493cccb56b78aefd503722e
64951c8c56b78aefd503724e
6496861b56b78aefd5037261
6497c07756b78aefd5037270
6499185a56b78aefd503728b
649a78cb56b78aefd50372a3
649bf38856b78aefd50372c1
649d492456b78aefd50372d2
649e78a656b78aefd50372e4
649fc42d56b78aefd50372fe
64a10cda56b78aefd5037313
64a29d8056b78aefd5037329
64a3b0c956b78aefd503733f
64a5435c56b78aefd5037367
64a679b656b78aefd503737b
64a7d92256b78aefd5037386
64a8e94856b78aefd503739b
64aa9dbd56b78aefd50373b9
64abad65b368511cc6c20692
64ace2b0b368511cc6c206a4
64ae2198b368511cc6c206ba
64afb43ab368511cc6c206d2
64b16525b368511cc6c206f6
64b29948b368511cc6c20705
64b3ed66b368511cc6c2071a
64b4ea54b368511cc6c2072c


0       None
2       None
3       None
4       None
5       None
        ... 
1723    None
1724    None
1726    None
1729    None
1730    None
Length: 749, dtype: object

In [39]:
# client["magazin"]["journals"].update_many({}, { "$set" :{
#   "branch._id":
#     "aadee444-306d-48e1-aa5c-501b86a584be"
# }})

UpdateResult({'n': 749, 'electionId': ObjectId('7fffffff000000000000017f'), 'opTime': {'ts': Timestamp(1750970928, 758), 't': 383}, 'nModified': 749, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1750970928, 758), 'signature': {'hash': b'N\xc7\r\xe9.\x8f\x9c1\xfd\x12\xf9p\x1c\xb7\xdc\x84\xda2\xb4\xa3', 'keyId': 7457592122544226512}}, 'operationTime': Timestamp(1750970928, 758), 'updatedExisting': True}, acknowledged=True)

In [68]:
all_journals = client["magazin"]["journals"].find()

In [69]:
all_journals = list(all_journals)

In [73]:
all_journals[15]

{'_id': ObjectId('64912ae356b78aefd50371f7'),
 'branch': {'name': 'Xonobod',
  'location': 'Xonobod',
  'phone': '+998 97 034 38 58',
  '_id': 'aadee444-306d-48e1-aa5c-501b86a584be'},
 'date': datetime.datetime(2023, 6, 20, 0, 0),
 'shift_is_closed': True,
 'terminal_income': 555000,
 'cash_left': 3450000,
 'total': 14076000,
 'operations': ['649145b456b78aefd50371f9',
  '64914df556b78aefd50371fa',
  '64914e0956b78aefd50371fb',
  '649155df56b78aefd50371fc',
  '64915f5256b78aefd50371fe',
  '6491888756b78aefd5037200',
  '649199bb56b78aefd5037204',
  '6491a40b56b78aefd5037205',
  '6491ae2056b78aefd5037206',
  '6491af2856b78aefd5037207',
  '6491cb9456b78aefd5037209',
  '6491cc6356b78aefd503720a',
  '6491cc9d56b78aefd503720b',
  '6491e29556b78aefd503720d']}

In [71]:
from pymongo.collection import ObjectId
for i in range(len(all_journals)):
    all_journals[i]["_id"] = ObjectId(all_journals[i]["_id"])


In [74]:
client["magazin"]["journals"].insert_many(
    all_journals
)

InsertManyResult([ObjectId('648179417ff10797db3b4bae'), ObjectId('648179467ff10797db3b4baf'), ObjectId('6481794a7ff10797db3b4bb0'), ObjectId('6481794d7ff10797db3b4bb1'), ObjectId('64829d078f196da12d8e98b6'), ObjectId('6483eded8f196da12d8e98d9'), ObjectId('64855c838f196da12d8e98fc'), ObjectId('6486a313c83c7f8bd5528b81'), ObjectId('6487efb3c83c7f8bd5528b9f'), ObjectId('6489590fc83c7f8bd5528bc5'), ObjectId('648a9cbfc83c7f8bd5528be3'), ObjectId('648bd86ec83c7f8bd5528bff'), ObjectId('648d3e2fc83c7f8bd5528c1c'), ObjectId('648e7877c83c7f8bd5528c33'), ObjectId('648fd56ec83c7f8bd5528c4d'), ObjectId('64912ae356b78aefd50371f7'), ObjectId('6492a71456b78aefd5037218'), ObjectId('6493cf4a56b78aefd5037230'), ObjectId('64957b4d56b78aefd5037257'), ObjectId('6496c18b56b78aefd5037266'), ObjectId('6498139f56b78aefd5037279'), ObjectId('6499368d56b78aefd503728c'), ObjectId('649a7fb856b78aefd50372a5'), ObjectId('649bb6ef56b78aefd50372b9'), ObjectId('649d0b2a56b78aefd50372cc'), ObjectId('649e629e56b78aefd50372

In [57]:
# Update all journals to convert string IDs to ObjectIDs
client["magazin"]["journals"].update_many({"branch.name": "Polevoy"}, [
    {
        "$set": {
            "_id": {
                "$toObjectId": "$_id"
            }
        }
    }
])


WriteError: After applying the update, the (immutable) field '_id' was found to have been altered to _id: ObjectId('64816df17ff10797db3b4ba8'), full error: {'index': 0, 'code': 66, 'errmsg': "After applying the update, the (immutable) field '_id' was found to have been altered to _id: ObjectId('64816df17ff10797db3b4ba8')"}